In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn

import optuna
import dagshub
import pickle

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [3]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        with open(f"{model_name}_model", "wb") as f:
            pickle.dump(model, f)

        with open(f"{model_name}_model", "rb") as f:
            clf = pickle.load(f)

        mlflow.log_artifact(f"{model_name}_model", "model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-08-26 21:07:59,313] A new study created in memory with name: no-name-2721b420-e475-499a-924e-5817712cafae
[I 2025-08-26 21:07:59,368] Trial 0 finished with value: 0.7030851063829787 and parameters: {'alpha': 0.8018010094356147}. Best is trial 0 with value: 0.7030851063829787.
[I 2025-08-26 21:07:59,383] Trial 1 finished with value: 0.7087234042553191 and parameters: {'alpha': 0.0005443821366788187}. Best is trial 1 with value: 0.7087234042553191.
[I 2025-08-26 21:07:59,395] Trial 2 finished with value: 0.7032978723404255 and parameters: {'alpha': 0.742708906022659}. Best is trial 1 with value: 0.7087234042553191.
[I 2025-08-26 21:07:59,404] Trial 3 finished with value: 0.7075531914893617 and parameters: {'alpha': 0.02220275364275526}. Best is trial 1 with value: 0.7087234042553191.
[I 2025-08-26 21:07:59,415] Trial 4 finished with value: 0.708404255319149 and parameters: {'alpha': 0.004130861917620541}. Best is trial 1 with value: 0.7087234042553191.
[I 2025-08-26 21:07:59,424]

🏃 View run MultinomialNB_SMOTE_TFIDF_Trigrams at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/0c4f8dfb81404c638371a7b6b8a647e9
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
